# How to Perform a Polymerase Chain Reaction (PCR)

> Visit the full library documentation [here](https://bjornfjohansson.github.io/pydna/)

This page walks you through how to perform a PCR, and design PCR primers, using `pydna`. A PCR amplifies a specific stretch of DNA defined by the primers, and it is critical to ensure primer binding specificity and appropriate primer melting temperature (tm) through careful design. `pydna` provides tools for quick modelling of PCR to check for correct PCR products, and methods for calculating tm and primer design, as performed in other popular web servcies (e.g Primer3Plus). 

## Modelling PCR with Known Primers

To perform PCR, `pydna` provides the `anneal` class and the `pcr` method to quickly generate expected primer products, on a `Dseqrecord` object. The `pcr` method needs only the forward and reverse primers, and the sequence. The primers must be passed from the 5' to the 3' end, following biological convention. More information on `Dseqrecord` and importing DNA sequences can be found in the other guide pages. 
  
The following example uses a 300+ bp custom sample circular DNA, containing an example gene that we would like to clone. 18 bp forward and reverse primers have been provided. 

In [ ]:
from pydna.amplify import pcr
from pydna.dseqrecord import Dseqrecord
from pydna.parsers import parse

#Importing GenBank file containing sample sequence 
path = "./sample_seq.gb"
record = parse(path)[0]

#Defining forward and reverse primers
fwd_primer = "ATTCCTGCAGAGTACATC"
rvs_primer = "ACCATCCGAAGATATCTT"

#Performing PCR
amplicon = pcr(fwd_primer, rvs_primer, record)

#Printing results
amplicon.format("gb")

LOCUS       45bp_PCR_prod             45 bp    DNA     linear   UNK 01-JAN-1980
DEFINITION  pcr_product_description_description.
ACCESSION   45bp
VERSION     45bp
KEYWORDS    .
SOURCE      .
  ORGANISM  .
            .
FEATURES             Location/Qualifiers
     gene            1..45
                     /label="example_gene"
     primer_bind     1..18
                     /label="name"
                     /PCR_conditions="primer sequence:ATTCCTGCAGAGTACATC"
                     /ApEinfo_fwdcolor="#baffa3"
                     /ApEinfo_revcolor="#ffbaba"
     primer_bind     complement(28..45)
                     /label="name"
                     /PCR_conditions="primer sequence:ACCATCCGAAGATATCTT"
                     /ApEinfo_fwdcolor="#baffa3"
                     /ApEinfo_revcolor="#ffbaba"
ORIGIN
        1 attcctgcag agtacatcaa ttctatgaag atatcttcgg atggt
//

The results of PCR are stored as a `Dseqrecord` object, with all the features inside of the amplicon included. Note how the example gene has been retained as a feature in `amplicon`. In addition, two new feature has been added to the record to indicate the forward and reverse primer binding region. 

`pydna` also allows modelling for PCR with extra bases on the 5' end of primers. This functionality is useful for modelling molecular cloning with multiple steps, where you might want to add different restriction sites to amplicons and ensure that that the right amplicons have been generated.  
  
For instance, to make sure that I can add a HindIII restriction site (AAGCTT)at the end of my `example_gene` without accidental hybridisation with other parts of the circular sequence, I can perform PCR using `pydna`. 

In [ ]:
fwd_primer = "AAGCTTATTCCTGCAGAGTACATC"
rvs_primer = "AAGCTTACCATCCGAAGATATCTT"

#Performing PCR
amplicon_HindIII = pcr(fwd_primer, rvs_primer, record)

#Printing results
amplicon_HindIII.format("gb")

LOCUS       57bp_PCR_prod             57 bp    DNA     linear   UNK 01-JAN-1980
DEFINITION  pcr_product_description_description.
ACCESSION   57bp
VERSION     57bp
KEYWORDS    .
SOURCE      .
  ORGANISM  .
            .
FEATURES             Location/Qualifiers
     primer_bind     1..21
                     /label="name"
                     /PCR_conditions="primer sequence:AAGCTTATTCCTGCAGAGTACATC"
                     /ApEinfo_fwdcolor="#baffa3"
                     /ApEinfo_revcolor="#ffbaba"
     gene            4..48
                     /label="example_gene"
     primer_bind     complement(31..48)
                     /label="name"
                     /PCR_conditions="primer sequence:AAGCTTACCATCCGAAGATATCTT"
                     /ApEinfo_fwdcolor="#baffa3"
                     /ApEinfo_revcolor="#ffbaba"
ORIGIN
        1 aagcttattc ctgcagagta catcaattct atgaagatat cttcggatgg taagctt
//

For more information on restriction digests and ligation, please refer to the Restriction and Ligation page.  

### Other ways of visualising the amplicons 

In addition to the normal `print` function and the `.format()` method (More information can be found in Dseq and Importing_Seqs pages, respectively), amplicons can also be visualized in other ways.

We can check the sequence of the amplicon alone using the `.seq` attribute on a PCR product:

In [ ]:
print(amplicon.seq)

ATTCCTGCAGAGTACATCAATTCTATGAAGATATCTTCGGATGGT


We can also visualize the amplicon as a visual map, using the `.map` method.

In [ ]:
print(amplicon.figure())

5ATTCCTGCAGAGTACATC...AAGATATCTTCGGATGGT3
                      ||||||||||||||||||
                     3TTCTATAGAAGCCTACCA5
5ATTCCTGCAGAGTACATC3
 ||||||||||||||||||
3TAAGGACGTCTCATGTAG...TTCTATAGAAGCCTACCA5


# Designing Primers and Calculating Tm in pydna

